In [1]:
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename
from pathlib import Path

### Functions

In [2]:
def to_camel_case(snake_str):
    components = snake_str.split('_')
    return ''.join(x.title() for x in components)

### Opening config .erv file

In [3]:
Tk().withdraw()
#filename = askopenfilename()
#filename = 'C:/Users/Matheus Ervilha/Google Drive/# PUC/13. TCC/Create Tables/SpotifyTables/inputs_spotify.erv'
filename = 'C:/Users/Matheus Ervilha/Google Drive/# PUC/13. TCC/Create Tables/SpotifyTables V2/inputs_spotify.erv'

with open(filename, 'r') as f:
    config = f.readlines()
    
final_path = '/'.join(filename.split('/')[:-1]) + '/'
    
Path(final_path + 'Script/').mkdir(parents=True, exist_ok=True)

In [4]:
filename

'C:/Users/Matheus Ervilha/Google Drive/# PUC/13. TCC/Create Tables/SpotifyTables V2/inputs_spotify.erv'

### Parsing File

In [5]:
actual_table = ''
tables = {}
for line_raw in config:
    line = line_raw.replace('\n', '').replace(' ', '')
        
    if line == '':
        actual_table = ''
        continue
    if actual_table == '':
        if '$' in line:
            main_table = line.replace('$', '')
            line = line.replace('$', '')

        actual_table = line
        tables[actual_table] = {}
        tables[actual_table]['items'] = []
        tables[actual_table]['names'] = []
        continue
    if actual_table != '':
        item = line.replace('\t', '').replace('    ', '')
        if '#' in item:
            tables[actual_table]['id'] = item.split(',')[0].replace('#', '')
        if '$' in item:
            tables[actual_table]['main'] = item.split(',')[0].replace('$', '')
        item = item.replace('$', '').replace('#', '')
        tables[actual_table]['items'].append(item)
        tables[actual_table]['names'].append(item.split(',')[0])

### Creating create_tables.sql file

In [6]:
create_tables = ''
for table in tables:
    actual_table = 'CREATE TABLE IF NOT EXISTS ' + table + ' ('
    
    foreign = []
    for i in range(0, len(tables[table]['items'])):
        item = tables[table]['items'][i]
        if 'ID' in item:
            item = item.replace('#', '')
            actual_table = actual_table + '\n\t' + item.split(',')[0] + ' SERIAL PRIMARY KEY,'
            continue
        actual_table = actual_table + '\n\t' + item.split(',')[0] + ' ' + item.split(',')[1]
        if i != len(tables[table]['items']) - 1:
            actual_table = actual_table + ','
        if len(item.split(',')) == 3:
            foreign_string = 'FOREIGN KEY (' + item.split(',')[0] + ') REFERENCES ' + item.split(',')[2].replace('(', ' (') + ','
            foreign.append(foreign_string)
    if len(foreign) > 0:
        actual_table = actual_table + ',\n\t'
    actual_table = actual_table + '\n\t'.join(foreign)[:-1]
    actual_table = actual_table + '\n);\n\n'
    create_tables = create_tables + '' + actual_table

file = open(final_path + "create_tables.sql","w")
file.write(create_tables)
file.close() 

### Creating Support Files
#### 1. RAMData.py

In [7]:
RAMData = open("./reference_files/RAMData.py","r").read()
declarations = ''
for table in tables:
    declarations = declarations + table + ' = {}\n    '

RAMData = RAMData.replace('$declarations', declarations)

RAMData_output = open(final_path + "Script/RAMData.py","w")
RAMData_output.write(RAMData)
RAMData_output.close() 

#### 2. SQLConn.py

In [8]:
SQLConn = open("./reference_files/SQLConn.py","r").read()
ids = ''
for table in tables:
    ids = ids + '\'' + table + '\': 0,\n        '

SQLConn = SQLConn.replace('$ids', ids)

SQLConn_output = open(final_path + "Script/SQLConn.py","w")
SQLConn_output.write(SQLConn)
SQLConn_output.close() 

print('!!! Remember to change database credentials !!!')

!!! Remember to change database credentials !!!


#### 3. SQLData.py

In [9]:
SQLData = open("./reference_files/SQLData.py","r").read()

# Declarations
declarations = ''
for table in tables:
    declarations = declarations + table + ' = []\n    '
SQLData = SQLData.replace('$declarations', declarations)

# Main_table
SQLData = SQLData.replace('$main_table', main_table)

# Save
save = ''
for table in tables:
    save = save + 'self.save_' + table + '()\n        '
SQLData = SQLData.replace('$save', save)

# Insert
insert = ''
for table in tables:
    insert = insert + '    def insert_' + table + '(self, ' + table + '):\n'
    insert = insert + '        self.' + table + '.append(' + table + ')\n'
    if table == main_table:
        insert = insert + '        self.checkMaxSize()\n'
    insert = insert + '\n'
SQLData = SQLData.replace('$insert', insert)

# Query
query = ''
for table in tables:
    query = query + '    def save_' + table + '(self):\n'
    query = query + '        valuesString = []\n'
    query = query + '        for new_' + table + ' in self.' + table + ':\n'
    s_array = []
    for i in range(0, len(tables[table]['names'])):
        if tables[table]['items'][i].split(',')[1] in ['INTEGER', 'DOUBLE', 'FLOAT', 'NUMERIC']:
            s_array.append('%s')
        else:
            s_array.append('\'%s\'')
    query = query + '            valuesString.append("""(' + (','.join(s_array)) + ')""" % ( \\\n'
    for i in range(0, len(tables[table]['names'])):
        if tables[table]['items'][i].split(',')[1] in ['INTEGER', 'DOUBLE', 'FLOAT', 'NUMERIC']:
            query = query + '                                new_' + table + '.get(\'' + tables[table]['names'][i] + '\', \'null\')'
        else:
            query = query + '                                new_' + table + '.get(\'' + tables[table]['names'][i] + '\', \'\')'
        if i != len(tables[table]['names']) - 1:
            query = query + ','
        query = query + '\n'
    query = query + '                            ))\n\n'
    query = query + '        tabela = \'' + table + '\'\n'
    query = query + '        columns = """\n'
    i_array = []
    for i in range(0, len(tables[table]['names'])):
        query = query + '            ' + tables[table]['names'][i]
        if i != len(tables[table]['names']) - 1:
            query = query + ','
        query = query + '\n'
    query = query + '        """\n'
    query = query + '        insertString = f\'INSERT INTO %s (%s) VALUES %s\' % (tabela, columns, (",".join(valuesString) ) )\n'
    query = query + '        if len(valuesString) == 0:\n'
    query = query + '            return\n'
    query = query + '        insertString = insertString.replace(\'\\\'null\\\'\', \'null\')\n'
    query = query + '        SQLConn.instance().cursor.execute(insertString)\n'
    query = query + '        self.' + table + ' = []\n'
    query = query + '\n\n'
    
    if tables[table].get('main', '') != '':
        query = query + '    def update_' + table + '(self, ' + table + '):\n'
        query = query + '        keys_raw = list(' + table + '.keys())\n'
        query = query + '        setValues = []\n'
        query = query + '        for key in keys_raw:\n'
        query = query + '            if key == \'' + table + '_id\':\n'
        query = query + '                continue\n'
        query = query + '            if isinstance(' + table + '[key], float) or isinstance(' + table + '[key], int):\n'
        query = query + '                setValues.append(key + \' = \' + str(' + table + '[key]))\n'
        query = query + '            else:\n'
        query = query + '                setValues.append(key + \' = \\\'\' + str(' + table + '[key]) + \'\\\'\')\n\n'
        query = query + '        if len(setValues) == 0:\n'
        query = query + '            return\n\n'
        query = query + '        updateString = f\'UPDATE ' + table + ' SET %s WHERE ' + table + '_id = %s\' % (\', \'.join(setValues), ' + table + '[\'' + table + '_id\'])\n'
        query = query + '        SQLConn.instance().cursor.execute(updateString)\n'
        query = query + '\n\n'
SQLData = SQLData.replace('$query', query)

SQLData_output = open(final_path + "Script/SQLData.py","w")
SQLData_output.write(SQLData)
SQLData_output.close() 

#### 4. All of the itens

In [10]:
SQLData = open("./reference_files/Item.py","r").read()

for table in tables:
    get = ''
    if tables[table].get('main', '') != '':
        get = get + 'def get' + to_camel_case(table) + 'Id(' + table + '):\n'
        get = get + '    ' + table + '_id = RAMData.instance().' + table + '.get(' + table + '[\'' + tables[table]['main'] + '\']' + ')\n'
        get = get + '    if ' + table + '_id is not None:\n'
        get = get + '        return ' + table + '_id\n\n'
        get = get + '    ' + table + '[\'' + table + '_id\'] = SQLConn.instance().getId(\'' + table + '\')\n'
        get = get + '    SQLData.instance().insert_' + table + '(' + table + ')\n'
        get = get + '    RAMData.instance().' + table + '[' + table + '[\'' + tables[table]['main'] + '\']] = ' + table + '[\'' + table + '_id\']\n'
        get = get + '    return ' + table + '[\'' + table + '_id\']\n\n'
        get = get + 'def update' + to_camel_case(table) + '(' + table + '):\n'
#         get = get + '    ' + table + '_id = RAMData.instance().' + table + '.get(' + table + '[\'' + tables[table]['main'] + '\']' + ')\n'
#         get = get + '    if ' + table + '_id is None:\n'
#         get = get + '        return get' + to_camel_case(table) + 'Id(' + table + ')\n\n'
        get = get + '    SQLData.instance().update_' + table + '(' + table + ')\n'
#         get = get + '    return ' + table + '[\'' + table + '_id\']\n\n'
    else:
        get = get + 'def save' + to_camel_case(table) + '(' + table + '):\n'
        get = get + '    code = RAMData.getCode(' + table + ')\n'
        get = get + '    ' + table + '_id = RAMData.instance().' + table + '.get(code)\n'
        get = get + '    if ' + table + '_id is not None:\n'
        get = get + '        return\n\n'
        get = get + '    ' + table + '[\'' + table + '_id\'] = SQLConn.instance().getId(\'' + table + '\')\n'
        get = get + '    SQLData.instance().insert_' + table + '(' + table + ')\n'
        get = get + '    RAMData.instance().' + table + '[code] = ' + table + '[\'' + table + '_id\']'
    
    item_data = SQLData.replace('$getsave', get)
    Item_output = open(final_path + 'Script/' + to_camel_case(table) + ".py","w")
    Item_output.write(item_data)
    Item_output.close() 
    

#### 5. Process File

In [11]:
Main = open("./reference_files/Main.py","r").read()

# Items
items = []
for table in tables:
    items.append(to_camel_case(table))
Main = Main.replace('$items', ', '.join(items))

# Tables
names = []
for table in tables:
    names.append(table)
Main = Main.replace('$tables', ','.join(names))

Main_output = open(final_path + "Main.py","w")
Main_output.write(Main)
Main_output.close() 

In [12]:
print('Done!')

Done!
